Библиотека Pandas — это весьма эффективный инструмент для обработки данных, представляющих собой временные ряды. На самом деле, эта библиотека была создана Уэсом МакКинни для работы с финансовыми данными, которые состоят, главным образом, из временных рядов.

При работе с временными рядами много сил уходит на выполнение различных операций с датой и временем. Этот материал посвящён ответам на четыре распространённых вопроса из данной сферы.

Возможно, вы уже сталкивались с этими вопросами. Ответить на все из них, кроме последнего, можно сравнительно просто. А вот ответ на последний, довольно-таки хитрый вопрос, представляет собой последовательность из нескольких действий.

Начнём с создания учебного датафрейма (объекта DataFrame), с которым будем экспериментировать:

In [23]:
import pandas as pd

In [24]:
df = pd.DataFrame({
    "booking_id": [1001, 1002, 1003, 1004, 1005],
    "property" : ["A", "A", "B", "B", "C"],
    "created_at": ["2022-03-01", "2022-02-10", "2022-04-12",
                   "2022-04-11", "2022-06-05"],
    "checkin_date": ["2022-06-01", "2022-06-10", "2022-06-02",
                     "2022-06-20", "2022-08-10"],
    "checkout_date": ["2022-06-06", "2022-06-15", 
                      "2022-06-06","2022-06-28", "2022-08-16"],
    "amount": [5400, 5600, 4800, 9000, 6500]
})
# изменение типа данных 
date_cols = ["created_at","checkin_date","checkout_date"]
df[date_cols] = df[date_cols].astype("datetime64[ns]")
# вывод объекта DataFrame
df

,booking_id,property,created_at,checkin_date,checkout_date,amount
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500


Для того чтобы у нас была бы возможность пользоваться функциями pandas, предназначенными для работы с датой и временем, даты в датафрейме должны быть представлены значениями подходящего типа данных. Именно по этой причине мы поменяли тип данных соответствующих столбцов на datetime64[ns].

1. Как извлечь из даты информацию о месяце и годе?
Дата содержит различные фрагменты информации, такие как год, день недели, месяц и так далее. Все эти фрагменты можно извлечь из даты с использованием методов, к которым обращаются через аксессор dt.

Например, месяц можно получить с помощью метода month. А как получить сведения о годе и месяце в формате ГГГГ-ММ? Ответ на этот вопрос уже не так очевиден. Извлечь эти сведения из даты можно, прибегнув к методу to_period:

In [25]:
# создание столбца year_month
df["year_month"] = df["created_at"].dt.to_period("M")
# вывод объекта DataFrame
df

,booking_id,property,created_at,checkin_date,checkout_date,amount,year_month
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400,2022-03
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600,2022-02
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800,2022-04
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000,2022-04
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500,2022-06


Как прибавить к дате временной интервал?
Прибавление к датам временных интервалов или вычитание их из дат — это одна из операций, часто встречающихся при работе с датами. Выполнить её можно, воспользовавшись функцией DateOffset.

Добавим 1 день к дате выселения (например — из некоего объекта недвижимости) (checkout_date) для записи о бронировании с идентификатором 1001:

In [26]:
df.loc[df["booking_id"]==1001, "checkout_date"] = \
df.loc[df["booking_id"]==1001, "checkout_date"] + \
pd.DateOffset(days=1)
# проверка результата
print(df.loc[df["booking_id"]==1001, "checkout_date"])
# вывод 0   2022-06-07
# Name: checkout_date, dtype: datetime64[ns]

0   2022-06-07
Name: checkout_date, dtype: datetime64[ns]


3. Как найти интервал в днях между двумя датами?
Интервал между двумя датами можно найти, вычтя одну из другой. Результатом этой операции будет объект Timedelta, а это не совсем то, что нам нужно:

In [27]:
df["checkout_date"][0] - df["checkin_date"][0]
# вывод Timedelta('6 days 00:00:00')

Timedelta('6 days 00:00:00')

Получить разницу между датами в днях, в виде целого числа, можно, воспользовавшись атрибутом days. Создадим столбец, который выводит разницу в днях между датой вселения (checkin_date) и датой создания бронирования (created_at):

In [28]:
# разница, выраженная в днях
df["days_to_checkin"] = \
(df["checkin_date"] - df["created_at"]).dt.days
# вывод объекта DataFrame
df

,booking_id,property,created_at,checkin_date,checkout_date,amount,year_month,days_to_checkin
0,1001,A,2022-03-01,2022-06-01,2022-06-07,5400,2022-03,92
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600,2022-02,120
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800,2022-04,51
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000,2022-04,70
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500,2022-06,66


4. Как сформировать последовательность дат между начальной и конечной датами?
Предположим, нам нужен календарь, который показывает дни, в которые объект недвижимости забронирован. Сейчас в первой строке датафрейма есть запись о том, что объект недвижимости A забронирован с 2022–06–01 по 2022–06–07. Получается, что объект A забронирован на даты 2022–06–01, 2022–06–02, 2022–06–03, 2022–06–04, 2022–06–05, 2022–06–06 (если предположить, что постоялец выселяется в 10 утра 2022–06–07).

Такой календарь можно создать, найдя даты между датами вселения и выселения и внеся их в объект DataFrame.

Для начала создадим датафрейм calendar, содержащий столбцы property, checkin_date и checkout_date:

In [29]:
# создание объекта DataFrame calendar
calendar = df[["property","checkin_date","checkout_date"]]

Функция date_range даёт нам даты, находящиеся между начальной и конечной датами. Вот что получится для первой записи о бронировании:

In [30]:
pd.date_range(calendar["checkin_date"][0], calendar["checkout_date"][0])
# вывод DatetimeIndex(['2022-06-01', '2022-06-02', '2022-06-03', 
#               '2022-06-04', '2022-06-05', '2022-06-06', 
#              '2022-06-07'],
#              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2022-06-01', '2022-06-02', '2022-06-03', '2022-06-04',
               '2022-06-05', '2022-06-06', '2022-06-07'],
              dtype='datetime64[ns]', freq='D')

Тут есть одна проблема: нам не нужно, чтобы дата выселения (checkout_date) была бы показана как дата, когда объект недвижимости забронирован. Поэтому мы, прежде чем находить диапазон дат, вычтем 1 из даты выселения.

Для того чтобы сделать это для всех строк, нужно воспользоваться функцией apply. Мы, кроме того, конвертируем вывод функции date_range в список, воспользовавшись конструктором list:

In [31]:
# создание столбца booked_days
calendar.loc[:, "booked_days"] = calendar.apply(
    
    lambda x: list(
        pd.date_range(
            x.checkin_date, 
            x.checkout_date - pd.DateOffset(days=1)
        ).date
    ),
    axis = 1    
)
# вывод объекта DataFrame
calendar

C:\Users\R5\AppData\Local\Temp\ipykernel_14504\2349268788.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calendar.loc[:, "booked_days"] = calendar.apply(


,property,checkin_date,checkout_date,booked_days
0,A,2022-06-01,2022-06-07,"[2022-06-01, 2022-06-02, 2022-06-03, 2022-06-0..."
1,A,2022-06-10,2022-06-15,"[2022-06-10, 2022-06-11, 2022-06-12, 2022-06-1..."
2,B,2022-06-02,2022-06-06,"[2022-06-02, 2022-06-03, 2022-06-04, 2022-06-05]"
3,B,2022-06-20,2022-06-28,"[2022-06-20, 2022-06-21, 2022-06-22, 2022-06-2..."
4,C,2022-08-10,2022-08-16,"[2022-08-10, 2022-08-11, 2022-08-12, 2022-08-1..."


Далее — нам нужно заполнить датафрейм, развернув даты из столбца booked_days. В этом нам поможет функция explode:

In [32]:
# функция explode 
calendar = calendar.explode(
    column="booked_days", ignore_index=True
)[["property","booked_days"]]
# вывод первых 5 строк
calendar.head()

,property,booked_days
0,A,2022-06-01
1,A,2022-06-02
2,A,2022-06-03
3,A,2022-06-04
4,A,2022-06-05


Теперь у нас имеется календарь дней, в которые объект недвижимости забронирован.

Итоги
Мы нашли ответы на 4 вопроса, решили 4 задачи, с которыми вы, скорее всего, столкнётесь, работая с временными рядами, содержащими даты. Надеемся, вам пригодится то, о чём вы сегодня узнали.

Добрый день! Метод to_datetime не работает с датами в формате 01.01.2022. Что делать с такими датами? Как их быстро конвертировать в 2022-01-01?

0


AnikinNN
22 июл 2022 в 19:36
У функции pd.to_datetime() есть аргумент format, в который можно передать формат даты в виде строки. В вашем примере это будет "%d.%m.%Y". Эта функция приводит строки к временнОму типу. Данные такого временного типа можно отображать разными способами, в том числе и с помощью такого же format

источник https://habr.com/ru/companies/wunderfund/articles/677068/ 